In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
def tileset_info():
    return {
        'min_pos': [-2.5, -2.5],
        'max_pos': [2.5, 2.5],
        'bins_per_dimension': 256,
        'max_width': 5,
        'max_zoom': 50,
        'mirror_tiles': 'false',
    }

In [13]:
from numba import jit, vectorize, guvectorize, float64, complex64, int32, float32
import numpy as np

# from: https://www.ibm.com/developerworks/community/blogs/jfp/entry/How_To_Compute_Mandelbrodt_Set_Quickly?lang=en

@jit(int32(complex64, int32))
def mandelbrot(c,maxiter):
    nreal = 0
    real = 0
    imag = 0
    for n in range(maxiter):
        nreal = real*real - imag*imag + c.real
        imag = 2* real*imag + c.imag
        real = nreal;
        if real * real + imag * imag > 4.0:
            return n
    return 0

@guvectorize([(complex64[:], int32[:], int32[:])], '(n),()->(n)')
def mandelbrot_numpy(c, maxit, output):
    #print("hellow")
    maxiter = maxit[0]
    for i in range(c.shape[0]):
        output[i] = mandelbrot(c[i],maxiter)

"""
import numexpr as ne

def mandelbrot_numpy(c, maxiter):
    output = np.zeros(c.shape)
    z = np.zeros(c.shape, np.complex64)
    for it in range(maxiter):
        notdone = ne.evaluate('z.real*z.real + z.imag*z.imag < 4.0')
        output[notdone] = it
        z = ne.evaluate('where(notdone,z**2+c,z)')
    output[output == maxiter-1] = 0    
    return output
  """      
def mandelbrot_set2(xmin,xmax,ymin,ymax,width=256,height=256,maxiter=500):
    #print("hi")
    r1 = np.linspace(xmin, xmax, width, dtype=np.float32)
    r2 = np.linspace(ymin, ymax, height, dtype=np.float32)
    c = r1 + r2[:,None]*1j
    n3 = mandelbrot_numpy(c,maxiter)
    return n3

#%timeit mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,256,256,100)
%time mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,256,256,100)
%time mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,1024,1024,100)
#%time mandelbrot_set2(-2.5, 2.5, -2.5, 2.5)

CPU times: user 31.2 ms, sys: 7.36 ms, total: 38.6 ms
Wall time: 67.6 ms
CPU times: user 479 ms, sys: 13.3 ms, total: 492 ms
Wall time: 498 ms


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [29]:
%%time 

import functools as ft
import numpy as np
import hgflask.server as hfse
import hgflask.tilesets as hfti
import hgflask.client as hgc
import hgtiles.format as hgfo
import hgtiles.utils as hgut
import os.path as op

# from https://plus.maths.org/content/computing-mandelbrot-set

def tiles(tsinfo, z, x, y):
    [from_x, from_y, to_x, to_y] = hgut.tile_bounds(tsinfo, z, x, y)
    tile_size = tsinfo['bins_per_dimension'] if 'bins_per_dimension' in tsinfo else 256
    limit = 4
    
    #print("from_x", from_x, to_x, from_y, to_y)
    #points = mandelbrot(from_x, from_y, to_x, to_y, tile_size)
    points = mandelbrot_set2(from_x, to_x, from_y, to_y)
    #print("points:", points)

    
    # ravel because the points are returned as a 2D array
    #return points.ravel()
    return hgfo.format_dense_tile(points.ravel())
    
tilesets = [{
    'uuid': 'a',
    'handlers': {
        'tileset_info': tileset_info,
        'tiles': ft.partial(hgut.tiles_wrapper_2d, 
                            tiles_function=ft.partial(tiles, tileset_info()))
    }},
    {
        'uuid': 'b',
        'filepath': op.expanduser('~/Dropbox/paper-data/gaia/cooler.mcool')
    }
]

ts = hfti.Tileset(
    uuid = 'a',
    tileset_info = tileset_info,
                 tiles=ft.partial(hgut.tiles_wrapper_2d, 
                            tiles_function=ft.partial(tiles, tileset_info())) )
# ts1 = hfti.cooler(uuid='b', filepath=op.expanduser('~/Dropbox/paper-data/gaia/cooler.mcool'))

server=hfse.start([ts])
#points = tiles(tsinfo, 1, 1, 1)

terminating: KcGTAMiETp-MpLXvLGW-SA


 * Running on http://0.0.0.0:58469/ (Press CTRL+C to quit)


sleeping


127.0.0.1 - - [18/Oct/2018 20:27:59] "GET /api/v1/tileset_info/?d=a HTTP/1.1" 200 -


ret: 200 b'{\n  "a": {\n    "bins_per_dimension": 256, \n    "max_pos": [\n      2.5, \n      2.5\n    ], \n    "max_width": 5, \n    "max_zoom": 50, \n    "min_pos": [\n      -2.5, \n      -2.5\n    ], \n    "mirror_tiles": "false"\n  }\n}\n'
returning
CPU times: user 25.1 ms, sys: 33.7 ms, total: 58.9 ms
Wall time: 267 ms


127.0.0.1 - - [18/Oct/2018 20:29:06] "OPTIONS /api/v1/tileset_info/?d=a&s=QiZ8jTkjQeO7eV2fvDFSYw HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2018 20:29:06] "GET /api/v1/tileset_info/?d=a&s=QiZ8jTkjQeO7eV2fvDFSYw HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2018 20:29:06] "OPTIONS /api/v1/tiles/?d=a.1.1.0&d=a.1.1.1&d=a.1.0.0&d=a.1.0.1&s=QiZ8jTkjQeO7eV2fvDFSYw HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2018 20:29:06] "GET /api/v1/tiles/?d=a.1.1.0&d=a.1.1.1&d=a.1.0.0&d=a.1.0.1&s=QiZ8jTkjQeO7eV2fvDFSYw HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2018 20:29:10] "OPTIONS /api/v1/tiles/?d=a.3.1.2&d=a.3.1.3&d=a.3.1.4&d=a.3.1.5&d=a.3.2.2&d=a.3.2.3&d=a.3.2.4&d=a.3.2.5&d=a.3.3.2&d=a.3.3.3&d=a.3.3.4&d=a.3.3.5&d=a.3.4.2&d=a.3.4.3&d=a.3.4.4&d=a.3.4.5&s=QiZ8jTkjQeO7eV2fvDFSYw HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2018 20:29:10] "GET /api/v1/tiles/?d=a.3.1.2&d=a.3.1.3&d=a.3.1.4&d=a.3.1.5&d=a.3.2.2&d=a.3.2.3&d=a.3.2.4&d=a.3.2.5&d=a.3.3.2&d=a.3.3.3&d=a.3.3.4&d=a.3.3.5&d=a.3.4.2&d=a.3.4.3&d=a.3.4.4&d=a.3.4.5&s=QiZ8jTkjQeO7eV2fv

In [15]:
!curl localhost:{server.port}/api/v1/tileset_info/?d={ts.uuid}

{
  "a": {
    "bins_per_dimension": 256, 
    "max_pos": [
      2.5, 
      2.5
    ], 
    "max_width": 5, 
    "max_zoom": 50, 
    "min_pos": [
      -2.5, 
      -2.5
    ], 
    "mirror_tiles": "false"
  }
}


In [16]:
server.process

<Process(Process-2, started)>

In [30]:
import json

conf = hgc.ViewConf([
    hgc.View([
        hgc.Track(tileset_uuid='a', 
               track_type='heatmap', 
               position='center', 
               api_url=server.api_address,
               height=400, options={
               }),
        hgc.Track(track_type='top-axis', position='top'),
        hgc.Track(track_type='left-axis', position='left')
    ])
])


#print(hgc.to_json_string())
import higlass_jupyter
higlass_jupyter.HiGlassDisplay(viewconf=conf.to_json(), 
                hg_options={"bounded": False, "renderer": "canvas"})

track.position center
track.position top
track.position left


A Jupyter Widget